
# RSCAM Computer Assessment 1


In this assessment you will use Python to solve some ordinary differential equations, to compute solutions numerically using discretization methods, to graph their solutions and to explore the behavior of differential equations and discretizations under changes in parameters.

Note that there are exercises embedded below, worth a total of 50 marks. After each exercise you will see a number in brackets that indicates the value of that exercise.

It is unlikely that you will complete the entire notebook in 2 hours.  Any remaining work can be done after the workshop.  Please check the learn web page for the assessment upload link and deadline.

After each graph that you produce you will wish to add a cell containing a brief comment. You are also asked to provide 'discussion' paragraphs in the notebook.  One way to do that is to use colour to highlight these observations, by embedding some markdown. "\<span style="color:blue">Observations: this is a comment.\<\/span\>".

At the start of any coding, we need a few commands to load relevant packages. We also introduce some abbreviations, "np" for numpy and "plt" for pyplot.

In [16]:
import numpy as np
import math
import matplotlib.pyplot as plt

# Comparing Euler and Leapfrog Methods on the Trimer
Euler's method computes an approximate trajectory of a differential equation.  We can easily compute numerical solutions using Euler's method for any differential equation.  If we do this for several different stepsizes, we get different discrete approximations to the solution trajectory.  Another numerical method, which is particular good for conservative systems, is the leapfrog scheme.   Here you will compare the two methods by implementing them and simulating the dynamics of a simple model, the Lennard-Jones trimer.

## The Lennard-Jones Trimer
The LJ trimer is a model for a system of three atoms interacting through a pairwise Lennard-Jones potential. 
$$
\varphi_{\rm LJ}(r) = r^{-12} - r^{-6}.
$$
By considering special 'isosceles' configurations of the three atoms, we can simplify the equations for the LJ Trimer to a model with just two degrees of freedom, as mentioned in the slides of Lecture 1.

The Hamiltonian is then
$$ H = \frac{p_x^2}{4} + \frac{9p_y^2}{4} + 2\varphi_{\rm LJ}\left (\sqrt{x^2+y^2}\right ) + \varphi_{\rm LJ}(2x)$$.
The kinetic energy is $$K(p) = \frac{p_x^2}{4} + \frac{9p_y^2}{4}$$.   This is in the form $K(p) = p^T M^{-1} p/2$ where the mass matrix is $$M = {\rm diag}(2, 2/9)$$.   The potential energy function is $$U(q) = 2\varphi_{\rm LJ}\left (\sqrt{x^2+y^2} \right ) + \varphi_{\rm LJ}\left (2x \right)$$.

**Exercise 1a [2 marks]**: write out and then code the equations of motion of the trimer from the Hamiltonian.  In Python there is a distinction between row and column vectors.  We will use column vectors.   Let $q = [x, y]^T$ and $p=[p_x, p_y]^T$.  You will need to code a small function to compute the force which is the **negative gradient** of the potential energy function.  The input of the force function is a 2d column vector q and the output is a 2d column vector F.  A template is provided below for you to fill in.

In [17]:
def phi_lj(r):
    y = r**(-12) - r**(-6)
    return y

In [18]:
def phi_lj_der(r):
    yp = -12 * r**(-13) + 6 * r**(-7)
    return yp

In [19]:
def trimer_force(q):
    # This function computes the negative gradient of the Trimer Potential function.
    
    # Split of x and y from the position vector q
    x = q[0]
    y = q[1]
    
    f = np.zeros(q.shape)
    
    r = np.sqrt(x**2 + y**2)
    phi_lj_der_r = phi_lj_der(r)
    
    der_wrt_x = 2 * phi_lj_der_r * x / r + 2 * phi_lj_der(2 * x)
    der_wrt_y = 2 * phi_lj_der_r * y / r
    
    f[0] = -der_wrt_x
    f[1] = -der_wrt_y
    
    return f

Next, we need to code the two methods using the same calling sequence.  This way we can easily compare them.  

**Exercise 1b [2 marks]**: Write a function euler_step that takes a pair of input (column) vectors q and p, and a stepsize h, and produces an updated pair of vectors Q,P which are obtained by applying a single step of the Euler method.

In [20]:
def euler_step(q, p, h, force_field=trimer_force):
    # Unpack the input vectors q and p
    q_x, q_y = q
    p_x, p_y = p

    # Compute the force at the current position
    f = force_field(q)

    # Update the position and momentum vectors using the Euler method
    Q = np.array([q_x + h * p_x, q_y + h * p_y])
    P = np.array([p_x + h * f[0], p_y + h * f[1]])

    return Q, P

The final piece of the puzzle is a routine which computes a sequence of steps, with a given numerical method, starting from a given initial condition.  The resulting trajectory is stored in a pair of matrices QQ and PP, where the columns of QQ are the position vectors at the successive timesteps and the columns of PP are the momentum vectors at successive timesteps.

**Exercise 1c [2 marks]:** Write a function compute_trajectory which takes as inputs an initial position vector q0, an initial momentum vector p0, a stepsize h, a function one_step (in the form of the code for euler_step), a function force_field (in the form of the trimer_force routine given above, and a number of steps, N.  The output of this routine should be two matrices with N+1 columns and 2 rows, one for positions and one for momenta. The first column of the position array is the initial condition.  The n+1st column is the position vector after step n. 

In [21]:
def compute_trajectory(q0, p0, h, N, one_step=euler_step, force_field=trimer_force):
    q_traj = q0
    p_traj = p0
    t_traj = [0]
    
    q, p, t = q0, p0, 0
    
    for n in range(0, N):
        q,p = one_step(q, p, h, force_field)
        t += h
        
        q_traj = np.column_stack((q_traj, q))
        p_traj = np.column_stack((p_traj, p))
        t_traj.append(t)
        
    return q_traj, p_traj

**Exercise 1d - accuracy comparisons [8 marks]:** implement the leapfrog method using the same approach as in Exercise 1b. 
For a system in the form $\dot{q} = p$, $\dot{p} = f(q)$, this method is given by
$$ \left| \begin{array}{rcl} 
p_{n+1/2}&:=&p_n + (h/2)\, f( q_n )\\
\qquad q_{n+1}&:=&q_n + h\, p_{n+1/2}\\
p_{n+1}&:=&p_{n+1/2} + (h/2)\, f( q_{n+1} )\\
\end{array}\right.$$
Using N steps of size h= 1/N, compare the accuracy of the two methods at time 1.   Use the same approach discussed in Lecture 3.  First produce an accurate "reference" or baseline solution using a very small stepsize (h=1e-5), then compare the results you obtain for larger stepsizes (1e-4, 1e-3, 1e-2) using each of the two methods.  You should consider different choices of the stepsize besides those mentioned in order to ensure that you see the asymptotic behavior for small stepsize).   All of the comparisons can be done at time 1.  **Use a loglog graph** to compare the error behaviors (where the x-axis is the stepsize in log scale and the y-axis is the error at time t=1).   Discuss the results (at most around 200 words are needed).

In [22]:
def leapfrog_step(q, p, h, force_field=trimer_force):
    # Compute the half step momentum update
    f = force_field(q)
    p_half = p + (h / 2) * f

    # Compute the full step position update
    q_next = q + h * p_half

    # Compute the force at the updated position
    f_next = force_field(q_next)

    # Compute the full step momentum update
    p_next = p_half + (h / 2) * f_next

    return q_next, p_next

In [24]:
# For the accuracy comparison:
def compute_error_at_time_1(h, one_step, q0, p0):
    N = int(1 / h)
    q_traj, p_traj = compute_trajectory(q0, p0, h, N, one_step=one_step)
    # Assuming the "reference" or baseline solution is obtained with h = 1e-5
    q_ref, p_ref = compute_trajectory(q0, p0, 1e-5, int(1 / 1e-5), one_step=one_step)
    error = np.linalg.norm(q_traj[:, -1] - q_ref[:, -1])  # Computing error at time 1
    return error

In [25]:
q0 = np.array([1, 1])
p0 = np.array([0, 0])

step_sizes = [1e-4, 1e-3, 1e-2]
errors_euler = [compute_error_at_time_1(h, euler_step, q0, p0) for h in step_sizes]
errors_leapfrog = [compute_error_at_time_1(h, leapfrog_step, q0, p0) for h in step_sizes]

plt.loglog(step_sizes, errors_euler, label='Euler Method')
plt.loglog(step_sizes, errors_leapfrog, label='Leapfrog Method')
plt.xlabel('Stepsize')
plt.ylabel('Error at time 1')
plt.legend()
plt.show()

ValueError: Integers to negative integer powers are not allowed.

**Exercise 1e - energy error comparisons [6 marks]:** Implement an energy function, trimer_energy, which computes the energy as a function of q and p.  You should ideally design this function so that you can pass the trajectory (QQ,PP) to the function and have it compute the energies at all the steps.   Compare the energy growth against time for the two methods (Euler, leapfrog).  In this case you should **graph the energy** as a function of time for each trajectory.  You may choose much larger ending times than t=1, for example t=10, t=100 etc.  Perform enough runs using different stepsizes to gain a good understanding of the energy behavior of each method.   Briefly discuss the graphs.


# Twin Pendulum

In this exercise you will modify the python code to compute solutions of a system of two particles moving in the plane. The particles will each be connected to the origin by a spring with rest length.  The particles also interact in a spring potential.

We think of our position vector $q$ as holding the position data for the whole system, so $q=[q_1,q_2]=[q_{1,x},q_{1,y},q_{2,x},q_{2,y}]$, and, similarly, $p$ is a vector in $\bf{R}^4$. We can write the new energy function as $$ H(q,p) = \| p_1 \|^2/2 + \| p_2 \|^2/2 + \omega_1^2(\|q_1\|-1)^2+ \omega_2^2(\|q_2\|-1)^2 + \epsilon \|q_1-q_2\|^2/2$$
Here $\omega_1,\omega_2$ are the frequencies of two nonlinear oscillators and $eps$ is the coupling coefficient.

**Exercise 2a [6]:**
Define new functions  `twinpend_energy` and `twinpend_force` using the trimer functions as models.  You do not need to compare the Euler and leapfrog methods in this example, **just use the leapfrog scheme.**

**Exercise 2b [6]:**
Set the constants $\omega_1,\omega_2$ equal to one.  Set $\epsilon=0.1$ to represent a moderate coupling.  Take as initial condition $q_0 = [0,1,0,1]^T$ and $p_0 = [1,-1,1,1]^T$.  You will need to experiment a little: choose the stepsize by trial and error to get an accurate approximation of the true motion, but bear in mind the need to use a relatively large stepsize (e.g. $h=0.1$) in order to compute a reasonably long-term evolution of the system. Plot the trajectories of the two particles in the $(x,y)$ plane, on the same axes, with a separate curve for each particle.  Separately, plot the energy as a function of time.  

**Exercise 2c - evaluation and discussion [8]:**
Using graphs and text, explore the motion of the twin pendulum system.    
Are the solutions periodic?  Bounded?  Is this  a chaotic dynamical system?  
Conduct further experiments to understand the system: what happens when you start them with different intial conditions?  With high velocities?  How does the choice of stepsize alter the behavior you observe?   Your answer for this part should comprise about one paragraph  (at most around 250 words) of English text, with a few graphs, all incorporated as markup in your jupyter notebook.

**Exercise 2d - energy conservation and adiabatic separation at long times [10]:**
In this exercise you will explore the conservation of total energy and the exchange of energy between two coupled pendula.  This related to the concept of adiabatic separation which plays an important role in physical models.

Define two energies by $E_1 = \| p_1 \|^2/2 + \omega_1^2(\|q_1\|-1)^2$ and $E_2 = \| p_2 \|^2/2 + \omega_2^2(\|q_2\|-1)^2$. Note that the total energy in the twin pendulum system is the sum of $E_1$ and $E_2$ plus the coupling term $\epsilon \|q_1-q_2\|^2/2$.  Create a function that takes QQ and PP and outputs arrays $E_1$, $E_2$ and $E$ which contain the corresponding energies at each step.

Set $\omega_1=\omega_2=1$ and $\epsilon =.1$.   Starting with the initial condition $q_0 = [0,-1,0,-1]^T$, $p_0 =[10,1,0,0]^T$, compute trajectories of the system and graph the trajectories in the $xy$-plane (separate curves for each particle). Use a stepsize $h=0.01$ and integrate for 1e5 steps.  Separately, plot the energies $E_1$ and $E_2$ over time.  Clearly the first subsystem starts with a higher initial energy than the second one ($E_1>>E_2$), but you should see that the energy gets exchanged rapidly between the two systems.  Despite this, you should also see that the total energy is well conserved, even when the time interval is fairly long.

Now consider the behavior of the system as $h$ is changed from $h=0.01$ to $h=0.1$, with all other parameters the same as before (N=1e5 steps).  (You can explore what happens when N is increased even further, if your computer is powerful enough!)  You should observe that the energy is no longer well conserved.  How does the total energy conservation depend on the stepsize, all other things being constant?

Try decreasing $\epsilon$ to 0.01 (reducing the strength of the coupling) and repeat your tests.    How does the strength of the coupling effect the results?

You may wish to explore what happens as you make the second frequency larger: $\omega_2=10$.

Your answer to this exercise should be around 500 words, maximum, supplemented by a few graphs.